# Installing the ComAp package
...

# Authentication - API secrets
To link the API to your account, and for security reasons, each API call needs to have two parameters: `ComAp-Key` and `Token`.  
To get the ComAp-Key, login to https://portal.websupervisor.net/developer (using your WSV Pro login credentials). There are two keys available - Primary and Secondary - pick one of them - it does not matter which one.

*(The idea is, that you use one of them in production, and the second for testig. After the testing, you can Regenerate the key, to make sure it stays secret. And this will not impact the other key that is used in production.)*

### Generating token
The `Token` is generated by calling API `Authenticate`. As part of this package, there is a script `get-token.py`, that will ask you for the `ComAp-Key`, your WSV username and password, generare the `Token` and store it in the config file. This script is called from the `setup.bat` script. But you can call it any time by running `python get-token` from the command line. 
Please do not share this configuration file, it contains your login information. 
The `Token` automatically expires after 1 month if it is not used. The `ComAp-Key` can be regenerated in case it gets leaked.

### Storing the API secrets 
The API key should not be hard-coded in the script. There are number of ways for storing the keys securely, that are beyond the scope of this document. For the purpose of this repository, we are storing the key in file `config.py`, with following content:
``` python
KEY='<YOUR KEY GOES HERE>'
TOKEN='<THE TOKEN GOES HERE>'
```
The individual scripts then get the stored secrets by calling:

In [2]:
from config import KEY,TOKEN 

# Calling WebSupervisor
All ComAp WebSupervisor APIs are methods of `wsv` class. To use them, you need to authenticate to an instance:

In [5]:
from comap.api import wsv

wsv=wsv(KEY,TOKEN)

Then you can use individual functions.

# Units
To get the list of units (gensets) under your WSV account, call method `wsv.units`.

It returns a list of units, each with the following attributes: `name`, `unitGuid`, `url`.

In [7]:
for unit in wsv.units():
    print(unit["name"])

IL3 AMF25 - nahradnik - GSM - test
Hybrid Barcelona
CGT Polyglass
IL-NT
ID FLX lite
Cogeneration unit ABB Prague
ID-Lite
IA-NT STD
IC-NT Mint
Cogeneration unit Hilton Longon
ID-EM
Harvester Goa
Habana hospital
IG-NT SPtM &#x28;CHP&#x29;
AIO GAS
New marine
IL9
Controller Parallel
Controller Solo
Hybrid
test33
sifrovani IL3
Real CHP IT
C07 - Hybrid Master
IG200_VAJL
